## Context

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store. Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales. Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.

## Objective

To build a predictive model and find out the sales of each product at a particular store. And then provide recommendations to the BigMart sales team to understand the properties of products and stores which play a key role in increasing sales.

## Data Dictionary

- **Item_Identifier** : Unique product ID

- **Item_Weight** : Weight of product

- **Item_Fat_Content** : Whether the product is low fat or not

- **Item_Visibility** : The % of the total display area of all products in a store allocated to the particular product

- **Item_Type** : The category to which the product belongs

- **Item_MRP** : Maximum Retail Price (list price) of the product

- **Outlet_Identifier** : Unique store ID

- **Outlet_Establishment_Year** : The year in which the store was established

- **Outlet_Size** : The size of the store in terms of ground area covered

- **Outlet_Location_Type** : The type of city in which the store is located

- **Outlet_Type** : Whether the outlet is just a grocery store or some sort of supermarket

- **Item_Outlet_Sales** : Sales of the product in the particular store. This is the outcome variable to be predicted.

We have two datasets - train (8523) and test (5681) data. The train dataset has both input and output variable(s). You need to predict the sales for the test dataset.

## Loading libraries and overview of datasets

In [1]:
# importing libraries for data manipulation
import numpy as np
import pandas as pd

# importing libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# importing libraries for building linear regression model
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# importing libraries for scaling the data
from sklearn.preprocessing import MinMaxScaler

#to ignore warnings
import warnings
warnings.filterwarnings("ignore")

#### Loading datasets

In [2]:
# loading both train and test datasets
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Train.csv'

In [ ]:
# checking the dataset
train_df.head()

#### Observations:

- As the variables `Item_Identifier` and `Outlet_Identifier` are only ID variables, we assume that they don't have any predictive power to predict the dependent variable / outcome variable - `Item_Outlet_Sales`.
- We remove these two variables from both the datasets.

In [ ]:
train_df = train_df.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)
test_df = test_df.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)

Now, let's find out some more information about the dataset i.e. total number of observations in the dataset, columns and their data types, etc.

In [ ]:
train_df.info()

#### Observations:

- The train datasets has `8523` number of observations and `10` columns. 
- Two columns- `Item_Weight` and `Outlet_Size` has missing values as their non-null values is less the total number of observations in the dataset. 
- We can also see that the columns - `Item_Fat_Content`, `Item_Type`, `Outlet_Size`, `Outlet_Location_Type` and `Outlet_Type` has data type `object` that means they are strings which means they are categorical variables. 
- Remaining all other variables are numerical in nature.

As we have already seen that two columns have missing values in the dataset. Let's check the percentage of missing values using the below piece of code.

In [ ]:
(train_df.isnull().sum()/train_df.shape[0])*100

Now we get the percentage of missing values for the columns - `Item_Weight` and `Outlet_Size` which are ~17% and ~28% respectively. We will see next how to treat these missing values.

## EDA and Data Preprocessing

Now that we have an understanding of the business problem which we want to solve and also we have loaded the datasets. The next step to follow is to have a better understanding of the dataset i.e. what is the distribution of the variables, what are different relationships that exist between variables, etc. If there is any data anomaly like missing values, outliers, how do we treat them to prepare the dataset for building the predictive model.

### Univariate Analysis

Let us now start exploring the dataset, by performing univariate analysis of the dataset i.e. analyzing/visualizing the dataset by taking one variable at a time. Data visualization is a very important skill to have and we need to decide what charts to plot to better understand the data. For example - what chart makes sense to analyze categorical variable or what charts are suited for numerical variable and also it depends on what relationship between variables we want to show?

Let's start with analyzing the `categorical` variables present in the data. There are five categorical variables in this dataset and we are creating univariate bar charts for each of them to check their distribution. 

Below we are creating 4 subplots to plot them in a single frame

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
  
fig.suptitle('Bar plot for all categorical variables in the dataset')
  

sns.countplot(ax=axes[0, 0], x='Item_Fat_Content', data=train_df, color='blue', 
              order=train_df['Item_Fat_Content'].value_counts().index, );

sns.countplot(ax=axes[0, 1], x='Outlet_Size', data=train_df, color='blue', 
              order=train_df['Outlet_Size'].value_counts().index);

sns.countplot(ax=axes[1, 0], x='Outlet_Location_Type', data=train_df, color='blue', 
              order=train_df['Outlet_Location_Type'].value_counts().index);

sns.countplot(ax=axes[1, 1], x='Outlet_Type', data=train_df, color='blue', 
              order=train_df['Outlet_Type'].value_counts().index);

#### Observations:

- From the above univariate plot for the variable `Item_Fat_Content`, it seems like there are errors in the data. The category - `Low Fat` is sometimes also written as `low fat` and `LF`. And also the category `Regular` is also written as `reg` sometimes. So **we need to fix this issue in the data**.
- For the column `Outlet_Size`, we can see that the most common category is - `Medium` followed by Small and High
- The most common category for the column `Outlet_Location_Type` is `Tier 3`, followed by Tier 2 and Tier 1. This makes sense now if we combine this information with the information on column `Outlet_Size`. We would expect High outlet size stores to be present in Tier 1 cities and the count of tier 1 cities is less so the count of high outlets size is also less. And we would expect more number of medium and small number of outlet sizes in the dataset because we have more number outlets present in tier 3 and tier 2 cities in the dataset.
- In the column `Outlet_Type`, the majority of the stores or the model is of `Supermarket Type 1` and we have less and almost equal number of representation in the other categories - Supermarket Type 2 and Supermarket Type 3 and Grocery Store.

Below we are analyzing the categorical variable `Item_Type`

In [ ]:
fig= plt.figure(figsize=(18, 6))
sns.countplot(x='Item_Type', data=train_df, color='blue', order=train_df['Item_Type'].value_counts().index);
plt.xticks(rotation=45);

From the above plot, we can see that majority of the the items sold in these stores are `Fruits and Vegetables`, followed by `Snack Foods`, `Household items`

Before we move ahead with the univariate analysis for the numerical variables, let's first fix the data issues that we have found out for the column `Item_Fat_Content`

In the below piece of code, we are replacing the categories - `low fat` and `LF` by `Low Fat` using lambda function and also we are replacing the category `reg` by Regular.

In [ ]:
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Low Fat' if x=='low fat' or x=='LF' else x)
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Regular' if x=='reg' else x)

The data preparation steps that we do on the training data, we need to perform the same steps on test data as well. So below we are performing the same transformation on `test` dataset - 

In [ ]:
test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].apply(lambda x: 'Low Fat' if x=='low fat' or x=='LF' else x)
test_df['Item_Fat_Content'] = test_df['Item_Fat_Content'].apply(lambda x: 'Regular' if x=='reg' else x)

Below we are analyzing all the `numerical` variables present in the data. And since we want to visualize one numerical variable at a time, histogram is the best choice to visualize the data

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
  
fig.suptitle('Histogram for all numerical variables in the dataset')
  
sns.histplot(x='Item_Weight', data=train_df, kde=True, ax=axes[0]);
sns.histplot(x='Item_Visibility', data=train_df, kde=True, ax=axes[1]);
sns.histplot(x='Item_MRP', data=train_df, kde=True, ax=axes[2]);

#### Observations:

- The variable `Item_Weight` is approx uniformly distributed and when we will impute the missing values for this column we will need to keep this in mind that we don't end up changing the distribution significantly after imputing those missing values
- The variable `Item_Visibility` is a right skewed distribution which means that there are certain items whose percentage of display area is much higher than the other items
- The variable `Item_MRP` is following approx multi-modal normal distribution

### Bivariate Analysis

Now let's move ahead with bivariate analysis and to understand how variables are related to each other and if there is a strong relationship between dependent and independent variables present in the training dataset. 

In the below plot - we are analyzing the variables `Outlet_Establishment_Year` and `Item_Outlet_Sales`. Here, since the variable `Outlet_Establishment_Year` is defining a time component, the best chart to analyze this relationship will be a line plot.

In [ ]:
fig= plt.figure(figsize=(18, 6))
sns.lineplot(x='Outlet_Establishment_Year', y='Item_Outlet_Sales', data=train_df, ci=None, estimator='mean');

#### Observations:

- The average sales are almost constant every year, but we don't see any increasing/decreasing trend in sales with time. So the variable year might not be a good predictor to predict sales, which we can see later in the modeling phase.
- Also, in the year 1998 the average sales has plummeted. This might be due to some external factors which are not included in the data.

Next, we are trying to find out linear correlations between the variables, This will help us to know which numerical variables are correlated with the dependent/target variable. Also we can find out multi-collinearity i.e. which pair of independent variables are correlated with each other.

In [ ]:
fig= plt.figure(figsize=(18, 6))
sns.heatmap(train_df.corr(), annot=True);
plt.xticks(rotation=45);

#### Observations:

- From the above plot, it seems only the independent variable `Item_MRP` has moderate linear relationship with the dependent variable `Item_Outlet_Sales`
- For the remaining, it does not seem like there is any strong positive/negative correlation between the variables

Next, we are creating the bivariate scatter plots to check relationships between pairs of independent and dependent variables 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
  
fig.suptitle('Bi-variate scatterplot for all numerical variables with the dependent variable')
  
sns.scatterplot(x='Item_Weight', y='Item_Outlet_Sales', data=train_df, ax=axes[0]);
sns.scatterplot(x='Item_Visibility', y='Item_Outlet_Sales', data=train_df, ax=axes[1]);
sns.scatterplot(x='Item_MRP', y='Item_Outlet_Sales', data=train_df, ax=axes[2]);

#### Observations:

- The first scatter plot shows the data is completely random and there is no relationship between `Item_Weight` and `Item_outlet_Sales`. This is also evident from the correlation number which we got above i.e. there is no strong correlation between these two variables
- The second scatter plot between `Item_Visibility` and `Item_outlet_Sales`, there is no good relationship between them. But we can see a pattern that, as the `Item_Visibility` increases from **0.19**, the sales decreases. This might be due the reason that the management has given more visibility to those items which are not generally sold often, thinking that better visibility would increase the sales. This information can also help us to engineer new feature like - a categorical variable with categories - `high visibility` and `low visibility`. But we are not doing this here. This is an idea, we may want to pursue in future.
- The third scatter plot between the variables - `Item_MRP` and `Item_outlet_Sales`, it is clear that there is a positive correlation between them and the variable `Item_MRP` would have a good predictive power to predict the sales.

### Missing value treatment

Here we are imputing missing values for the variable `Item_Weight`. There are many ways to impute missing values, we can impute the missing values by its `mean`, `median` and using advanced imputation algorithm like `knn` etc. But here we are trying to find out some relationship of the variable `Item_Weight` with other variables in the dataset to impute those missing values

And also after imputing the missing values for the variables, the overall distribution of the variable should not change significantly

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(train_df.pivot_table(index='Item_Fat_Content', columns='Item_Type', values='Item_Weight'), annot=True);
plt.xticks(rotation=45);

#### Observations:

- In the above heatmap, we can see that based on different combination of `Item_Types` and `Item_Fat_Content`, the average range of values for the column `Item_Weight` lies between the minimum value 10 and maximum value of 14

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(train_df.pivot_table(index='Outlet_Type', columns='Outlet_Location_Type', values='Item_Weight'), annot=True);
plt.xticks(rotation=45);

#### Observations:

- In the above heatmap, we can see that based on different combination of `Outlet_Type` and `Outlet_Location_Type`, the average range of values for the column `Item_Weight` is constant at 13

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(train_df.pivot_table(index='Item_Fat_Content', columns='Outlet_Size', values='Item_Weight'), annot=True);
plt.xticks(rotation=45);

#### Observations:

- In the above heatmap, we can see that based on different combination of `Item_Fat_Content` and `Outlet_Size`, the average range of values for the column `Item_Weight` is also constant at 13

We will impute the missing values using an uniform distribution with parameters a=10 and b=14, as shown below - 

In [ ]:
item_weight_indices_to_be_updated = train_df[train_df['Item_Weight'].isnull()].index
train_df.loc[item_weight_indices_to_be_updated, 'Item_Weight'] = np.random.uniform(10, 14, 
                                                                                   len(item_weight_indices_to_be_updated))

Performing the same transformation on `test` dataset

In [ ]:
item_weight_indices_to_be_updated = test_df[test_df['Item_Weight'].isnull()].index
test_df.loc[item_weight_indices_to_be_updated, 'Item_Weight'] = np.random.uniform(10, 14, 
                                                                                   len(item_weight_indices_to_be_updated))

Next we will be imputing missing values for the column `Outlet_Size`. Below are creating two different datasets - one where we have non-null values for the column `Outlet_Size` and in the other dataset we all the values of the column `Outlet_Size` are missing. We then check the distribution of the other variables in the dataset where `Outlet_Size` is missing to identify if there is any pattern present or not

In [ ]:
outlet_size_data = train_df[train_df['Outlet_Size'].notnull()]
outlet_size_missing_data = train_df[train_df['Outlet_Size'].isnull()]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
  
fig.suptitle('Bar plot for all categorical variables in the dataset where the variable Outlet_Size is missing')
  

sns.countplot(ax=axes[0], x='Outlet_Type', data=outlet_size_missing_data, color='blue', 
              order=outlet_size_missing_data['Outlet_Type'].value_counts().index, );

sns.countplot(ax=axes[1], x='Outlet_Location_Type', data=outlet_size_missing_data, color='blue', 
              order=outlet_size_missing_data['Outlet_Location_Type'].value_counts().index);

sns.countplot(ax=axes[2], x='Item_Fat_Content', data=outlet_size_missing_data, color='blue', 
              order=outlet_size_missing_data['Item_Fat_Content'].value_counts().index);

#### Observations:

- We can see that, in the dataset, wherever `Outlet_Size` is missing majority of them have `Outlet_Type` as Supermarket Type 1 and `Outlet_Location_Type` as Tier 2 and `Item_Fat_Content` as `Low Fat`

Now are creating a crosstab of all the above categorical variables against the column `Outlet_Size` where we want to impute the missing values

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(pd.crosstab(index=outlet_size_data['Outlet_Type'], columns=outlet_size_data['Outlet_Size']), annot=True, fmt='g')
plt.xticks(rotation=45);

#### Observations: 

- We can see from the crosstab, all the grocery stores have `Outlet_Size` as small
- And all the Supermarket Type 2 and Supermarket Type 3 have `Outlet_Size` as Medium

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(pd.crosstab(index=outlet_size_data['Outlet_Location_Type'], columns=outlet_size_data['Outlet_Size']), annot=True, 
            fmt='g')
plt.xticks(rotation=45);

#### Observations:

- We can see from the crosstab, all the Tier 2 stores have `Outlet_Size` as Small

In [ ]:
fig= plt.figure(figsize=(18, 3))
sns.heatmap(pd.crosstab(index=outlet_size_data['Item_Fat_Content'], columns=outlet_size_data['Outlet_Size']), annot=True, 
            fmt='g')
plt.xticks(rotation=45);

#### Observations:

- There is no clear pattern between the variables `Item_Fat_Content` and `Outlet_Size` unlike what we have got against `Outlet_Type` and `Outlet_Location_Type`

Now we will use the patterns we have from the variables `Outlet_Type` and `Outlet_Location_Type` to impute the missing values for the column `Outlet_Size`

Below we are identifying the indices in the dataframe where `Outlet_Size` is null/missing and `Outlet_Type` is Grocery Store, so that we can replace those missing values with the value Small based on the pattern we have identified in above crosstab visualization. Similarly we are also identifying the indices in the dataframe where `Outlet_Size` is null/missing and `Outlet_Location_Type` is Tier 2, to impute those missing values.

In [ ]:
grocery_store_indices = train_df[train_df['Outlet_Size'].isnull()].query("Outlet_Type == 'Grocery Store'").index
tier_2_indices = train_df[train_df['Outlet_Size'].isnull()].query("Outlet_Location_Type == 'Tier 2'").index

Now are updating those indices for the column `Outlet_Size` with the value Small

In [ ]:
train_df.loc[grocery_store_indices, 'Outlet_Size'] = 'Small'
train_df.loc[tier_2_indices, 'Outlet_Size'] = 'Small'

Performing the same transformation on `test` dataset

In [ ]:
grocery_store_indices = test_df[test_df['Outlet_Size'].isnull()].query("Outlet_Type == 'Grocery Store'").index
tier_2_indices = test_df[test_df['Outlet_Size'].isnull()].query("Outlet_Location_Type == 'Tier 2'").index

In [ ]:
test_df.loc[grocery_store_indices, 'Outlet_Size'] = 'Small'
test_df.loc[tier_2_indices, 'Outlet_Size'] = 'Small'

After we have imputed the missing values, let's check again if we still have missing values in both train and test datasets 

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

As we can see that there is no more missing values in the datasets

### Feature engineering

Now that, we have completed the data understanding and data preparation step, before starting the modeling task, we think that there are certain features which are not present in the dataset, but we can create using the existing columns, which we think can have the predictive power to predict the sales. This step of creating a new feature from the existing features in the dataset is known as **Feature Engineering**. So we will start with a hypothesis - As the store gets older, the sales increases. Now how do we define old? We know the establishment year and this data is collected in 2013, so the age can be found by subtracting establishment year from 2013. This is what we are doing in the below piece of code-

We are creating the a new feature `Outlet_Age` which indicates that how old the outlet is.

In [ ]:
train_df['Outlet_Age'] = 2013 - train_df['Outlet_Establishment_Year']
test_df['Outlet_Age'] = 2013 - test_df['Outlet_Establishment_Year']

In [ ]:
fig= plt.figure(figsize=(18, 6))
sns.boxplot(x='Outlet_Age', y='Item_Outlet_Sales', data=train_df);

**Observations:**
* The hypothesis that we had - **As the store gets older, the sales increases** does not seem to hold true based on the above plot. Because for different age of stores the sales have similar distribution approximately. But let's keep this variable as of now, and we will revisit this variable again at the time of model building and we remove this variable by observing its significance later on.

## Modeling

Now, we have analyzed the all the variables in the dataset, we are now ready to start building the model. We have observed that not all the independent variables are important to predict the outcome variable. But at the beginning, we will use all the variables and then from the model summary we will take decision which variable to remove from the model. Model building is an iterative task.

In [ ]:
# we are removing the outcome variable from the feature set and also the variable Outlet_Establishment_Year as we have created
# a new variable Outlet_Age
train_features = train_df.drop(['Item_Outlet_Sales', 'Outlet_Establishment_Year'], axis=1)

# and then we are extracting the outcome variable separately
train_target = train_df['Item_Outlet_Sales']

In linear based models, whenever we have categorical variables as independent variables, we need to create **one hot encoded** representation (which is also known as dummy variables) of those categorical variables. The below piece of code is creating dummy variables and we are removing the first category in those variables which is known as **reference variable**. The reference variable helps to interpret the linear regression which we will see later.

In [ ]:
# in linear based models it is mandatory to create dummy variables for the categorical variables
train_features = pd.get_dummies(train_features, drop_first=True)
train_features.head()

#### Observations:

- Notice the column names of all the categorical variables, and also the overall number of columns has increased after we created the dummy variables
- And for each of those categorical variables, the first category have been removed, e.g. - the category `Low Fat` of the categorical variable `Item_Fat_Content` have been removed which became the `reference variable` and we only have the category `Regular` as a new column `Item_Fat_Content_Regular`

Below we are scaling the numerical variables in the dataset to have same range. If we don't do this, then the model will be biased towards a variable where we have a higher range and the model will not learn from the variables which values with lower range. There are many ways to do scaling, here we are using `MinMaxScaler` as we have both categorical and numerical variables in the dataset and we don't want to change the dummy encodings of the categorical variables that we have already created. For more information on different ways of doing scaling, refer to the **section 6.3.1** of this page [here](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)

In [ ]:
# creating an instance of the MinMaxScaler
scaler = MinMaxScaler()

# applying fit_transform on the training features data
train_features_scaled = scaler.fit_transform(train_features)

# the above scaler returns the data in array format, below we are converting back to pandas dataframe
train_features_scaled = pd.DataFrame(train_features_scaled, index=train_features.index, columns=train_features.columns)
train_features_scaled.head()

Now as the dataset is ready and prepared, we are set to build the model using `statsmodels` package

In [ ]:
# here we are adding the intercept term
train_features_scaled = sm.add_constant(train_features_scaled)

# calling the OLS algorithm on the train features and target variable
ols_model_0 = sm.OLS(train_target, train_features_scaled)

# fitting the model
ols_res_0 = ols_model_0.fit()

Now let's observe summary of the model

In [ ]:
print(ols_res_0.summary())

- We can see that `R-squared` for the model is `0.563`. 
- Not all the variables are statistically significant to predict the outcome variable. To check which are statistically significant or have predictive power to predict the target variable, we need to check the `p-value` against all the independent variables.

**Interpreting the Regression Results:**

1. **Adjusted. R-squared**: It reflects the fit of the model.
    - R-squared values range from 0 to 1, where a higher value generally indicates a better fit, assuming certain conditions are met.
    - In our case, the value for Adj. R-squared is **0.562**

2. **coeff**: It represents the change in the output Y due to a change of one unit in the variable (everything else held constant).
3. **std err**: It reflects the level of accuracy of the coefficients.
    - The lower it is, the more accurate the coefficients are.
4. **P >|t|**: It is p-value.
   
   * Pr(>|t|) : For each independent feature there is a null hypothesis and alternate hypothesis 

    Ho : Independent feature is not significant 
   
    Ha : Independent feature is significant 
    
   * A p-value of less than 0.05 is considered to be statistically significant.

   
5. **Confidence Interval**: It represents the range in which our coefficients are likely to fall (with a likelihood of 95%).

To understand in detail, how p-values can help to identify statistically significant variables to predict the sales, we need to understand the hypothesis testing framework here - 

In the following example, we are showing the null hypothesis between the independent variable `Outlet_Size` and the dependent variable `Item_Outlet_Sales` to identify if there is any relationship between them or not.

- **Null hypothesis:** There is nothing going on or there is no relationship between variables `Outlet_Size` and `Item_Outlet_Sales` 

<center>$\mu_{\text {outlet size }=\text { High }}=\mu_{\text {outlet size }=\text { Medium }}=\mu_{\text {outlet size }=\text { Small }}$</center>

- **Alternate hypothesis:** There is something going on, because of which there is a relationship between variables `Outlet_Size` and `Item_Outlet_Sales` i.e. - 

<center>$\mu_{\text {outlet size = High }} \neq \mu_{\text {outlet size = Medium }} \neq \mu_{\text {outlet size }=\text { small }}$</center>

and from the above model summary if p-value less than the significance level 0.05, that means we will reject the null hypothesis in favor of alternate hypothesis. In other words, we have enough statistical evidence that there is relationship exists between the variables `Outlet_Size` and `Item_Outlet_Sales`

Based on this above analysis, if we observe the above model summary, we can see only some variables or some categories of a categorical variables have p-value lower than 0.05

### Feature Selection

As we have discussed above, that not all variables are statistically significant to predict the dependent/outcome variable. In this step, we are selecting only the relevant features `based on p-value` which we discussed above and eliminating all the other variables that are not important.

In [ ]:
# selecting only relevant features
train_features_2 = train_df[['Item_MRP', 'Outlet_Age', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]

Again we are creating the dummy variables for the categorical variables as we discussed above

In [ ]:
train_features_2 = pd.get_dummies(train_features_2, drop_first=True)
train_features_2.head()

Then we are scaling the variables to have the same scale

In [ ]:
train_features_scaled_2 = scaler.fit_transform(train_features_2)
train_features_scaled_2 = pd.DataFrame(train_features_scaled_2, index=train_features_2.index, columns=train_features_2.columns)
train_features_scaled_2.head()

And now build the model again, but this time with only taking the relevant variables based on p-values

In [ ]:
train_features_scaled_2 = sm.add_constant(train_features_scaled_2)

ols_model_2 = sm.OLS(train_target, train_features_scaled_2)
ols_res_2 = ols_model_2.fit()

In [ ]:
print(ols_res_2.summary())

**Observations**
- The R-Squared value did not change at all. It is still coming out to be 0.563 which implies that all other variables were not adding any value to the model.

Lets' check assumptions of the linear regression model

### Checking for assumptions and rebuilding the model

In this step, we will check for the below assumptions in the model, to check if they hold true or not. And if there is any issue, then we will rebuild the model after fixing those issues

1. No multicollinearity among independent variables
2. Mean of residuals should be 0
3. Normality of error terms
4. Linearity of variables
5.  No heteroscedasticity

#### No multicollinearity among independent variables

* Multicollinearity occurs when predictor variables in a regression model are correlated. This correlation is a problem because predictor variables should be independent.  If the correlation between variables is high, it can cause problems when we fit the model and interpret the results. When we have multicollinearity in the linear model, the coefficients that the model suggests are unreliable.

* There are different ways of detecting (or testing) multi-collinearity, one such way is Variation Inflation Factor.

* **Variance  Inflation  factor**:  Variance  inflation  factors  measures  the  inflation  in  the variances of the regression parameter estimates due to collinearities that exist among the  predictors.  It  is  a  measure  of  how  much  the  variance  of  the  estimated  regression coefficient βk is “inflated” by  the  existence  of  correlation  among  the  predictor variables in the model. 

* General Rule of thumb: If VIF is 1 then there is no correlation among the kth predictor and the remaining predictor variables, and  hence  the variance of β̂k is not inflated at all. Whereas if VIF exceeds 5 or is close to exceeding 5, we say there is moderate VIF and if it is 10 or exceeding 10, it shows signs of high multi-collinearity.

In [ ]:
# we drop the one with the highest vif value and check the Adjusted-R Squared
vif_series_1 = pd.Series([variance_inflation_factor(train_features_scaled_2.values,i) for i in range(train_features_scaled_2.shape[1])],
                        index=train_features_scaled_2.columns)
print('Series before feature selection: \n\n{}\n'.format(vif_series_1))

#### Removing Multicollinearity
 * To remove multicollinearity, we will drop the column that has VIF score greater than 5 and build the model again.

We can see that the variable `Outlet_Size_Medium` has the highest VIF, so in the next iteration we will remove this variable and build the model again

In [ ]:
train_features_scaled_3 = train_features_scaled_2.drop(['Outlet_Size_Medium'], axis=1)
ols_model_3 = sm.OLS(train_target, train_features_scaled_3)
ols_res_3 = ols_model_3.fit()

In [ ]:
print(ols_res_3.summary())

Now, let's check the VIF again

In [ ]:
vif_series_2 = pd.Series([variance_inflation_factor(train_features_scaled_3.values,i) for i in range(train_features_scaled_3.shape[1])],
                        index=train_features_scaled_3.columns)
print('Series before feature selection: \n\n{}\n'.format(vif_series_2))

- VIF score for `Outlet_Type_Supermarket Type2` is the highest and more than 5. Let's remove this variable and build the model again.

In [ ]:
train_features_scaled_4 = train_features_scaled_3.drop(['Outlet_Type_Supermarket Type2'], axis=1)
ols_model_4 = sm.OLS(train_target, train_features_scaled_4)
ols_res_4 = ols_model_4.fit()

In [ ]:
print(ols_res_4.summary())

In [ ]:
vif_series_3 = pd.Series([variance_inflation_factor(train_features_scaled_4.values,i) for i in range(train_features_scaled_4.shape[1])],
                        index=train_features_scaled_4.columns)
print('Series before feature selection: \n\n{}\n'.format(vif_series_3))

**Observations**
- We have removed the multi-collinearity from the data as all the variables have VIF score less than 5 (except constant)
- But some of the variables in the model have become insignificant. Let's drop these variables.

In [ ]:
train_features_scaled_5 = train_features_scaled_4.drop(['Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3'], axis=1)
ols_model_5 = sm.OLS(train_target, train_features_scaled_5)
ols_res_5 = ols_model_5.fit()

In [ ]:
print(ols_res_5.summary())

**Observations**
- All the variables are significant now and the R-Squared has not decreased by much. It is still coming out to be 0.55.

Let's check other assumptions

### Mean of residuals should be 0 and normality of error terms

In [ ]:
residual = ols_res_5.resid # Residuals

In [ ]:
residual.mean()

The mean of residuals is very close to 0. Hence, the corresponding assumption is satisfied.

### Test for Normality

**What is the test?**

* Error terms/Residuals should be normally distributed

* If the error terms are non- normally distributed, confidence intervals may become too wide or narrow. Once confidence interval becomes unstable, it leads to difficulty in estimating coefficients based on minimization of least squares.

**What do non-normality indicate?**

* It suggests that there are a few unusual data points which must be studied closely to make a better model.

**How to Check the Normality?**

* It can be checked via QQ Plot. Residuals following normal distribution will make a straight line plot otherwise not.

* Other test to check for normality : Shapiro-Wilk test.

**What is the residuals are not-normal?**

* We can apply transformations like log, exponential, arcsinh, etc as per our data

In [ ]:
# Plot histogram of residuals
sns.histplot(residual, kde=True)

We can see that the error terms are normally distributed. The assumption of normality is satisfied.

### Linearity of variables

It states that the predictor variables must have a linear relation with the dependent variable.

To test the assumption, we'll plot residuals and fitted values on a plot and ensure that residuals do not form a strong pattern. They should be randomly and uniformly scattered on the x-axis.

In [ ]:
# predicted values
fitted = ols_res_5.fittedvalues

# sns.set_style("whitegrid")
sns.residplot(x = fitted, y = residual, color="lightblue")
plt.xlabel("Fitted Values")
plt.ylabel("Residual")
plt.title("Residual PLOT")
plt.show()

**Observations**
- We can see that there is some pattern in fitted values and residuals i.e. the residuals are not randomly distributed.
- Let's try to fix this. We can apply the log transformation on the target variable and try to build a new model.

In [ ]:
#Log transformation on the target variable
train_target_log = np.log(train_target)

In [ ]:
#Fitting new model with the transformed target variable
ols_model_6 = sm.OLS(train_target_log, train_features_scaled_5)
ols_res_6 = ols_model_6.fit()

In [ ]:
# predicted values
fitted = ols_res_6.fittedvalues
residual1 = ols_res_6.resid

# sns.set_style("whitegrid")
sns.residplot(x = fitted, y = residual1, color="lightblue")
plt.xlabel("Fitted Values")
plt.ylabel("Residual")
plt.title("Residual PLOT")
plt.show()

**Observations**
- We can see that there is no pattern in the residuals vs fitted values scatter plot now i.e. the linearity assumption is satisfied.
- Let's check the model summary of the latest model we have fit. 

In [ ]:
print(ols_res_6.summary(0))

- The model performance has improved significantly. The R-Squared has increased from 0.55 to 0.675.

Let's check the final assumption

### No heteroscedasticity

#### Test for homoscedasticity

* homoscedasticity - If the variance of the residuals are symmetrically distributed across the regression line, then the data is said to homoscedastic.

* heteroscedasticity - If the variance is unequal for the residuals across the regression line, then the data is said to be heteroscedastic. In this case the residuals can form an arrow shape or any other non symmetrical shape.

We will use Goldfeld–Quandt test to check homoscedasticity.

Null hypothesis : Residuals are homoscedastic

Alternate hypothesis : Residuals are hetroscedastic

In [ ]:
from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(train_target_log, train_features_scaled_5)
lzip(name, test)

- As we can see from the above test the p-value is greater than 0.05, so we fail to reject the null-hypothesis. That means - residuals are homoscedastic.

We have verified all the assumptions of the linear regression model. The final equation of the model is as follows:

**$\log ($ Item_Outlet_Sales $)$ $=6.1604 + 1.9623 *$ Item_MRP - $1.0049 *$ Outlet_Age $ - 0.5812 *$ Outlet_Size_Small $+1.2249 *$ Outlet_Type_Supermarket Type1 $ + 1.9662 *$ Outlet_Type_Supermarket Type3**

Now we finalized with the model, let's prepare the test dataset to predict the output

In [ ]:
without_const=train_features_scaled_2.iloc[:,1:]
test_features = pd.get_dummies(test_df, drop_first=True)
test_features=test_features[list(without_const)]
test_features_scaled = scaler.transform(test_features)
test_features_scaled = pd.DataFrame(test_features_scaled, columns=without_const.columns)
test_features_scaled = sm.add_constant(test_features_scaled)
test_features_scaled.head()

In [ ]:
test_features_scaled = test_features_scaled[train_features_scaled_5.columns]

### Evaluation Metric

#### R-squared

The R-squared metric gives us an indication that how good/bad our model is from a baseline model. Here, we have explained ~98% variance in the data as compared to the baseline model when there is no independent variable.

In [ ]:
print(ols_res_6.rsquared)

#### Mean Squared Error

This metric measures the average of the squares of the errors i.e. the average squared difference between the estimated values and the actual value.

In [ ]:
print(ols_res_6.mse_resid)

#### Root Mean Squared Error

This metric is same as the above but instead of square root of MSE to get RMSE

In [ ]:
print(np.sqrt(ols_res_6.mse_resid))

Below we are checking the cross validation score to identify if the model that we have built is `underfitted`, `overfitted` or `just right` model.

In [ ]:
#Fitting linear model

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

linearregression = LinearRegression()                                    

cv_Score11 = cross_val_score(linearregression, train_features_scaled_5, train_target_log, cv = 10)
cv_Score12 = cross_val_score(linearregression, train_features_scaled_5, train_target_log, cv = 10, 
                             scoring = 'neg_mean_squared_error')                                  


print("RSquared: %0.3f (+/- %0.3f)" % (cv_Score11.mean(), cv_Score11.std() * 2))
print("Mean Squared Error: %0.3f (+/- %0.3f)" % (-1*cv_Score12.mean(), cv_Score12.std() * 2))

**Observations:**
- The R-Squared on the cross validation is 0.673 which is almost similar to the R-Squared on the training dataset.
- And the MSE on cross validation is 0.337 which is almost similar to the R-Squared on the training dataset.

It seems like that our model is `just right`. It is giving a generalized performance.

Since this model that we have developed is a linear model, which is not capable of capturing non-linear patterns in the data, so we may want to build more advanced regression model which can capture the non-linearities in the data and improve this model further. But that is out of scope for this case study.

### Predicting on the test dataset

Once our model is completed, we can now use this to predict the sales in our test data as shown below:

In [ ]:
# this test predictions will be on a log scale
test_predictions = ols_res_6.predict(test_features_scaled)

# we are converting the log scale predictions to its original scale
test_predictions_inverse_transformed = np.exp(test_predictions)

Point to remember: The output of this model is in log scale. So after prediction we need to transform this value in log back to its original scale by doing inverse of log transformation i.e. taking exponentiation</font>

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 6))

sns.histplot(test_predictions, ax=ax[0]);
sns.histplot(test_predictions_inverse_transformed, ax=ax[1]);

### Conclusion and Recommendations

- We performed EDA - univariate and bivariate analysis on all the variables in the dataset
- We then performed missing values treatment using relationship between variables
- We started the model building the process with all the features
- We analyzed the model summary report
- Then we checked for different assumptions of linear regression
- And we then fixed the model iteratively if any assumptions did not hold true
- Then we evaluated the model using different evaluation metrics

Lastly below is the model equation:

**$\log ($ Item_Outlet_Sales $)$ $= 6.1604 + 1.9623 *$ Item_MRP - $1.0049 *$ Outlet_Age $ - 0.5812 *$ Outlet_Size_Small $+1.2249 *$ Outlet_Type_Supermarket Type1 $ + 1.9662 *$ Outlet_Type_Supermarket Type3**

- From this above equation, we can interpret that - with one unit change in the variable `Item_MRP` the outcome variable log of `Item_Outlet_Sales` increases by 1.9623 units. So if we want to increase sales we may want to store higher MRP items in the high visibility area.

- On an average, the log sales of stores with outlet size small is 0.5812 less than the log sales of outlet size high (remember the **reference variable**, here for the variable outlet size - **the category high was the reference variable which we dropped at the time of creating the dummy variable**). So when we are interpreting the coefficients of categorical variables we need to compare them against its reference variable. So the recommendation to the management would be to focus more on the larger stores. May be they can focus on having more inventory in larger stores as compared to others as we don't want to loose on revenue because unavailability of items there.

- On an average, the log sales of stores with type Supermarket type 1 is 1.2249 more than the log sales of stores with type grocery stores (the **reference variable** for this categorical variable **Outlet_Type**, here is **grocery store**). And similarly for Supermarket Type 3, on an average, have 1.9662 times more log sales than grocery stores.

So after interpreting this linear regression equation it is clear that large stores of supermarket type 3 have more sales than others. So we want to maintain or improve the sales in these stores. And for the remaining ones we may want to make strategies to improve the sales e.g. providing better customer service, better training for store staffs, providing more visibility of high MRP item, etc.

In [4]:
model_1_features = ['sqft_living', 'bathrooms', 'bedrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

model_1 = linear_model.LinearRegression()
model_1.fit(train_data[model_1_features], train_data['price'])

model_2 = linear_model.LinearRegression()
model_2.fit(train_data[model_2_features], train_data['price'])

model_3 = linear_model.LinearRegression()
model_3.fit(train_data[model_3_features], train_data['price'])

# extracting the coef
print(model_1.coef_)
print(model_2.coef_)
print(model_3.coef_)

NameError: name 'linear_model' is not defined